In [ ]:
import numpy as np
import pandas as pd
import os
import math
import tensorflow as tf
import matplotlib.pyplot as plt

from tcn import TCN, tcn_full_summary
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Bidirectional, LSTM, Conv1D, MaxPooling1D, MultiHeadAttention, Flatten ,Reshape, TimeDistributed, Concatenate
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [ ]:
# dirs
DATA_DIR = "./load.csv"
TEST_PLOT_DIR = "./test_plots/TCN_Attention_BiLSTM/"

In [ ]:
if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)
if not os.path.exists("./model"):
    os.makedirs("./model")
if not os.path.exists("./training_history"):
    os.makedirs("./training_history")

In [ ]:
data = pd.read_csv(DATA_DIR)
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y/%m/%d %H:%M')
data['change_rate'] = data['Load'].pct_change()
data['change_rate'][0] = 0

In [ ]:
data['change_rate']

In [ ]:
# scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data['Load'].to_numpy().reshape(-1, 1))
data['Load'] = data_scaled

In [ ]:
# Define a function to generate a list of timestamps every 2 hours within the dataset's range
def generate_timestamps(data) -> pd.DatetimeIndex:
    start = data['Timestamp'].min() + DateOffset(days=3)
    end = data['Timestamp'].max() - DateOffset(hours=6, minutes=0)
    timestamps = pd.date_range(start=start, end=end, freq='15min')
    return timestamps


In [ ]:
timestamps = generate_timestamps(data)
print(timestamps)

In [ ]:
def generate_sets_for_all_timestamps(timestamps, data):
    training_sets = []
    change_rate_sets = []
    target_sets = []
    training_sets_time = []
    target_sets_time = []

    for timestamp in timestamps:
        # Calculate the range for the current period's data
        start_time_current = timestamp - DateOffset(days=2, hours=23, minutes=45)
        end_time_current = timestamp

        # Calculate the equivalent timestamp for last week
        # timestamp_last_week = timestamp - DateOffset(days=7)
        # start_time_last_week = timestamp_last_week - DateOffset(days=0, hours=23, minutes=45)
        # end_time_last_week = timestamp_last_week

        # Calculate the target range (the next 10 steps after the current timestamp)
        target_start_time = timestamp + DateOffset(minutes=15)
        target_end_time = timestamp + DateOffset(hours=6, minutes=0) 

        # Filter the data for training and target sets
        current_data = data[(data['Timestamp'] >= start_time_current) & (data['Timestamp'] <= end_time_current)]
        # last_week_data = data[(data['Timestamp'] >= start_time_last_week) & (data['Timestamp'] <= end_time_last_week)]
        target_data = data[(data['Timestamp'] >= target_start_time) & (data['Timestamp'] <= target_end_time)]

        # Combine current and last week data for the training set
        training_data = pd.concat([current_data]).reset_index(drop=True)
        
        # Save the training and target sets
        if not training_data.empty and not target_data.empty:
            training_sets.append(training_data['Load'])
            target_sets.append(target_data['Load'])
            training_sets_time.append(list(training_data['Timestamp']))
            target_sets_time.append(list(target_data['Timestamp']))

    training_sets = np.array(training_sets)
    target_sets = np.array(target_sets)
    training_sets_time = np.array(training_sets_time)
    target_sets_time = np.array(target_sets_time)

    return training_sets, target_sets, training_sets_time, target_sets_time


In [ ]:
# Generate training and target sets for all the timestamps
training_sets, target_sets, training_sets_time, target_sets_time = generate_sets_for_all_timestamps(timestamps, data)

In [ ]:
MONTH_TIME_STEP = math.floor(timestamps.shape[0] / 24)
X_test = []
y_test = []
X_test_time = []
y_test_time = []
minList = []
maxList = []
for i in range(0, 24):
    min = (i+1)*MONTH_TIME_STEP-(192*(i+1))
    max = (i+1)*MONTH_TIME_STEP-(192*i)
    X_test.append(training_sets[min:max])
    y_test.append(target_sets[min:max])
    X_test_time.append(training_sets_time[min:max])
    y_test_time.append(target_sets_time[min:max])
    training_sets = np.concatenate([training_sets[:min], training_sets[max:]])
    target_sets = np.concatenate([target_sets[:min], target_sets[max:]])
    training_sets_time = np.concatenate([training_sets_time[:min], training_sets_time[max:]])
    target_sets_time = np.concatenate([target_sets_time[:min], target_sets_time[max:]])


In [ ]:
X_test = np.concatenate([i for i in X_test])
y_test = np.concatenate([i for i in y_test])
X_test_time = np.concatenate([i for i in X_test_time])
y_test_time = np.concatenate([i for i in y_test_time])

In [ ]:
X_train = training_sets
y_train = target_sets
X_train_time = training_sets_time
y_train_time = target_sets_time

subsequences = 6
timesteps = X_train.shape[1] // subsequences

X_train_DL = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_DL = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)
print(np.array(X_train_DL).shape)
print(np.array(X_test_DL).shape)

In [ ]:
def create_model(input_shape, num_outputs):
    inputs = Input(shape=input_shape)

    # TCN layer
    x = TCN(activation="tanh", dilations=(1, 2, 4), padding='causal', return_sequences=True)(inputs)

    # Multihead Attention
    x = MultiHeadAttention(num_heads=3, key_dim=96)(x, x, x)

    # BiLSTM layer for understanding the temporal dependencies
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(50, return_sequences=False))(x)
    # A Dense layer for output
    dense1 = Dense(32, activation="linear")(x)
    outputs = Dense(num_outputs, activation='linear')(dense1)  # For regression output

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = create_model(input_shape=X_train_DL.shape[1:], num_outputs=24)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
with tf.device("/GPU:0"):
    history = model.fit(
        X_train_DL,
        y_train,
        verbose=1,
        epochs=100,
        batch_size=288,
    )

plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_test_DL)

mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("-" * 86)
print(f'mse: {mse:.4f}')
print(f'rmse: {rmse:.4f}')
print(f'mae: {mae:.4f}')
print(f'mape: {mape:.4f}')
print("-" * 86)

# --------------------------------------------------------------------------------------
# mse: 0.0055
# rmse: 0.0742
# mae: 0.0468
# mape: 0.1265
# --------------------------------------------------------------------------------------

In [ ]:
pred_data = scaler.inverse_transform(y_pred)
actual_data = scaler.inverse_transform(y_test)
previous_data = scaler.inverse_transform(X_test)
for i in range(actual_data.shape[0]):
    plt.figure(figsize=(12, 6))
    X1 = np.concatenate((X_test_time[i][-30:], y_test_time[i]))
    y1 = np.concatenate((previous_data[i][-30:], actual_data[i]))
    X2 = y_test_time[i]
    y_p = pred_data[i]
    y_a = actual_data[i]
    Xh = np.full(30, X1[len(X1)-24])
    yh = np.arange(0, 30, 1)
    plt.title(f"Time Series {i+1} prediction")
    plt.plot(X1, y1, '--', color='#98afc7')
    plt.plot(X2, y_p, label='Predict')
    plt.plot(X2, y_a, label='Actual')
    plt.scatter(X2, y_p)
    plt.scatter(X2, y_a)
    plt.plot(Xh, yh, color='#4863a0', alpha=0.5)
    plt.ylim(0, 25)
    plt.xlabel('Time step')
    plt.ylabel('Usage (kWh)')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.savefig(TEST_PLOT_DIR+f"Time_Series_{i+1}.png")
    plt.close()